In [1]:
import torch
from    torch import nn
from    torch.nn import functional as F
import  numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import os

# Team Libraries
# Needed in order to use modules from a directory level higher
import sys
sys.path.append('../')

from meta_utils import *
from Learner import Learner
from learner_config import *
from Meta import Meta
from MiniImagenet import MiniImagenet
from poison import *

<Figure size 432x288 with 0 Axes>

In [8]:
model = Learner(miniImgNet_config,3,84)
maml = Meta(miniImgNet_config)
pth = os.getcwd()
model_loc = "../models/Leaner_OBJ_MiniImagenet_n_way_5_k_shot_5_acc_0.60546875_dateMMDDYYY_10272021"
model.load_state_dict(model_loc, map_location=torch.device('cpu'))
maml.net = model
# model_small.train()

TypeError: load_state_dict() got an unexpected keyword argument 'map_location'

In [ ]:
mini_test = MiniImagenet('miniimagenet/', mode='test', n_way=5, k_shot=5,
                         k_query=15,
                         batchsz=100, resize=84)

In [ ]:
from random import randrange
import  numpy as np

torch.manual_seed(222)
torch.cuda.manual_seed_all(222)
np.random.seed(222)

db_test = DataLoader(mini_test, 1, shuffle=True, num_workers=0, pin_memory=True)
accs_all_test = []
clean_accs_all_test = []
poison_accs_all_test = []
i = 0
base_and_target_imgs = {}
display = False
for x_spt, y_spt, x_qry, y_qry in db_test:
    if display:
      # plot support set
      show_grid(x_spt, 'Support Set', 5)
      # plot query set
      show_grid(x_qry, 'Query Set', 15, fig_dim=[25, 25])
    x_spt, y_spt, x_qry, y_qry = x_spt.squeeze(0), y_spt.squeeze(0), \
                                 x_qry.squeeze(0), y_qry.squeeze(0)
    # Base line accuracy
    spt_idx = np.argsort(y_spt.numpy())[0]
    qry_idx = np.argsort(y_qry.numpy())[-1]
    base = x_spt[spt_idx]
    target = x_qry[qry_idx]
    x_qry = target
    y_qry = y_qry[qry_idx]
    x_qry = x_qry.reshape(1, 3, 84, 84)
    y_qry = y_qry.reshape(1)
    clean_accs = maml.finetuning(x_spt, y_spt, x_qry, y_qry)
    clean_accs_all_test.append(clean_accs)
    ############## ADD POISONS TO SUPPORT HERE ##############   
    # spt_idx = np.argsort(y_spt.numpy())[0]
    # qry_idx = np.argsort(y_qry.numpy())[-1]
    # base = x_spt[spt_idx]
    # target = x_qry[qry_idx]
    x_spt[spt_idx] = poison_image_watermark(base, target, .8)
    # x_qry = target
    # y_qry = y_qry[qry_idx]
    poison_accs = maml.finetuning(x_spt, y_spt, x_qry, y_qry)
    poison_accs_all_test.append(poison_accs)
    
    #  base_and_target_imgs[x_spt[support_idx]] = x_qry[query_idx]
    # print(base_and_target_imgs)
    # accs = maml.finetuning(x_spt, y_spt, x_qry, y_qry)
    # accs_all_test.append(accs)
    
    i += 1
    print(i, " clean: ", clean_accs[-1])
    print(i, " poison: ", poison_accs[-1])
total_clean_accs = np.array(clean_accs_all_test).mean(axis=0).astype(np.float16)
total_poison_accs = np.array(poison_accs_all_test).mean(axis=0).astype(np.float16)
print("Total clean accs: " + str(total_clean_accs))
print("total_poison_accs: " + str(total_poison_accs))